In [1]:
# -*- coding: utf-8 -*-
# 코드 내부에 한글을 사용가능 하게 해주는 부분입니다.

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import np_utils
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = pd.read_csv('../dataset/train_SectionDataset.csv', delimiter=",")
docs = data.title
classes = np_utils.to_categorical(data.section)

print('첫번째 학습셋 입력: ' + docs[0])

test = pd.read_csv('../dataset/test_SectionDataset.csv', delimiter=",")
test_docs = test.title
test_classes = np_utils.to_categorical(test.section)

print('첫번째 테스트셋 입력: ' + test_docs[0])
print('첫번째 학습셋 결과 one-hot 출력' + str(classes[0]))
print('첫번째 테스트셋 결과 one-hot 출력' + str(test_classes[0]))

doc_append = docs.append(test_docs) #X_train + X_test 수행



# 토큰화
token = Tokenizer()
token.fit_on_texts(doc_append) #인덱스 부여
x = token.texts_to_sequences(docs)
x_test = token.texts_to_sequences(test_docs)

word_size = len(token.word_index)
print('전체 데이터셋 단어 토큰 개수: ' + str(word_size))

print('첫번째 학습셋 토큰 결과: ' + str(x[0]))

print('첫번째 테스트셋 토큰 결과: ' + str(x_test[0]))

max_title = -1
for i in range(len(x)):
    if max_title < len(x[i]):
        max_title = len(x[i])
       
print('학습셋 제목 최대 길이: ' + str(max_title))

max_test_title = -1
for j in range(len(x_test)):
    if max_test_title < len(x_test[j]):
        max_test_title = len(x_test[j])
               
print('테스트셋 제목 최대 길이: ' + str(max_test_title))

#패딩
padded_x = sequence.pad_sequences(x, maxlen=18)
padded_test_x = sequence.pad_sequences(x_test, maxlen=18)

print('첫번째 패딩 토큰: ' + str(padded_x[0]))
print('첫번째 패딩 토큰 토큰' + str(padded_test_x[0]))

# 모델의 설정
model = Sequential()
model.add(Embedding(word_size + 1, 18))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding='valid', activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(4))
model.add(Activation('softmax'))
model.summary()

# 모델의 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델의 실행
history = model.fit(padded_x, classes, batch_size=100, epochs=5, validation_data=(padded_test_x, test_classes))

# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(padded_test_x, test_classes)[1]))


# 테스트 셋의 오차
#y_vloss = history.history['val_loss']

# 학습셋의 오차
#y_loss = history.history['loss']

# 그래프로 표현
#x_len = numpy.arange(len(y_loss))
#plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
#plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
#plt.legend(loc='upper right')
#plt.grid()
#plt.xlabel('epoch')
#plt.ylabel('loss')
#plt.show()


Using TensorFlow backend.


첫번째 학습셋 입력: 유시민 검찰도 사법도 썩었지 vs 진중권 유시민 망상 대중은 현실로 믿어
첫번째 테스트셋 입력: 박준영 “도자기 집에서 사용” 김선교 “궁궐 살았나”
첫번째 학습셋 결과 one-hot 출력[1. 0. 0. 0.]
첫번째 테스트셋 결과 one-hot 출력[1. 0. 0. 0.]
전체 데이터셋 단어 토큰 개수: 23319
첫번째 학습셋 토큰 결과: [243, 7314, 7315, 7316, 122, 413, 243, 7317, 7318, 4360, 4361]
첫번째 테스트셋 토큰 결과: [3139, 22912, 1069, 22913, 22914, 22915, 22916]
학습셋 제목 최대 길이: 18
테스트셋 제목 최대 길이: 14
첫번째 패딩 토큰: [   0    0    0    0    0    0    0  243 7314 7315 7316  122  413  243
 7317 7318 4360 4361]
첫번째 패딩 토큰 토큰[    0     0     0     0     0     0     0     0     0     0     0  3139
 22912  1069 22913 22914 22915 22916]
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 18)          419760    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 18)          0         
_____________________________________________________

C:\Users\shbk0\AppData\Local\conda\conda\envs\tutorial\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 10000 samples, validate on 150 samples
Epoch 1/5
10000/10000 [==============================] - 3s 275us/step - loss: 1.3734 - accuracy: 0.2989 - val_loss: 1.4129 - val_accuracy: 0.2400
Epoch 2/5
10000/10000 [==============================] - 2s 207us/step - loss: 1.0635 - accuracy: 0.5008 - val_loss: 1.3105 - val_accuracy: 0.3333
Epoch 3/5
10000/10000 [==============================] - 2s 180us/step - loss: 0.7299 - accuracy: 0.6917 - val_loss: 1.3549 - val_accuracy: 0.4467
Epoch 4/5
10000/10000 [==============================] - 2s 186us/step - loss: 0.5306 - accuracy: 0.7943 - val_loss: 1.5144 - val_accuracy: 0.4000
Epoch 5/5
150/150 [==============================] - 0s 60us/step

 Test Accuracy: 0.4067


In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import np_utils
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = pd.read_csv('../dataset/train_SectionDataset.csv', delimiter=",")
docs = data.title
classes = np_utils.to_categorical(data.section)

print('첫번째 학습셋 입력: ' + docs[0])

test = pd.read_csv('../dataset/test_SectionDataset.csv', delimiter=",")
test_docs = test.title
test_classes = np_utils.to_categorical(test.section)

print('첫번째 테스트셋 입력: ' + test_docs[0])
print('첫번째 학습셋 결과 one-hot 출력' + str(classes[0]))
print('첫번째 테스트셋 결과 one-hot 출력' + str(test_classes[0]))

doc_append = docs.append(test_docs) #X_train + X_test 수행



# 토큰화
token = Tokenizer()
token.fit_on_texts(doc_append) #인덱스 부여
x = token.texts_to_sequences(docs)
x_test = token.texts_to_sequences(test_docs)

word_size = len(token.word_index)
print('전체 데이터셋 단어 토큰 개수: ' + str(word_size))

print('첫번째 학습셋 토큰 결과: ' + str(x[0]))

print('첫번째 테스트셋 토큰 결과: ' + str(x_test[0]))

max_title = -1
for i in range(len(x)):
    if max_title < len(x[i]):
        max_title = len(x[i])
       
print('학습셋 제목 최대 길이: ' + str(max_title))

max_test_title = -1
for j in range(len(x_test)):
    if max_test_title < len(x_test[j]):
        max_test_title = len(x_test[j])
               
print('테스트셋 제목 최대 길이: ' + str(max_test_title))

#패딩
padded_x = sequence.pad_sequences(x, maxlen=18)
padded_test_x = sequence.pad_sequences(x_test, maxlen=18)

print('첫번째 패딩 토큰: ' + str(padded_x[0]))
print('첫번째 패딩 토큰 토큰' + str(padded_test_x[0]))

Using TensorFlow backend.


첫번째 학습셋 입력: 유시민 검찰도 사법도 썩었지 vs 진중권 유시민 망상 대중은 현실로 믿어
첫번째 테스트셋 입력: 박준영 “도자기 집에서 사용” 김선교 “궁궐 살았나”
첫번째 학습셋 결과 one-hot 출력[1. 0. 0. 0.]
첫번째 테스트셋 결과 one-hot 출력[1. 0. 0. 0.]
전체 데이터셋 단어 토큰 개수: 23319
첫번째 학습셋 토큰 결과: [243, 7314, 7315, 7316, 122, 413, 243, 7317, 7318, 4360, 4361]
첫번째 테스트셋 토큰 결과: [3139, 22912, 1069, 22913, 22914, 22915, 22916]
학습셋 제목 최대 길이: 18
테스트셋 제목 최대 길이: 14
첫번째 패딩 토큰: [   0    0    0    0    0    0    0  243 7314 7315 7316  122  413  243
 7317 7318 4360 4361]
첫번째 패딩 토큰 토큰[    0     0     0     0     0     0     0     0     0     0     0  3139
 22912  1069 22913 22914 22915 22916]


In [3]:
print(padded_x)

[[    0     0     0 ...  7318  4360  4361]
 [    0     0     0 ...  7319  7320  1168]
 [    0     0     0 ...  1375  2400  7325]
 ...
 [    0     0     0 ... 22908 22909   792]
 [    0     0     0 ... 22910 22911  5382]
 [    0     0     0 ...  3104    75   779]]
